# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
    
    for graph in P1_Graphs:
        
        degrees = graph.degree()
        degree_values = sorted(set(degrees.values()))
        histogram = [list(degrees.values()).count(i) for i in degree_values]

        #plt.plot(degree_values, histogram, '-')
        #plt.xlabel('Degrees')
        #plt.ylabel('Fraction of Nodes')
        #plt.show()

        #print(nx.average_clustering(graph), nx.average_shortest_path_length(graph), nx.number_of_nodes(graph))
        
        ## First and fourth ones exhibit power law; are 'PA' models.        
        ## Second and fifth have lower average_shortest_path_lengths; 'SW_H' models.
        ## Third one have high average_shortest_path_lengths and average_clustering; 'SW_L' model.
    return ['PA','SW_L','SW_L','PA','SW_H']

graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [4]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, roc_curve, auc

def salary_predictions():
    
    df = pd.DataFrame(index=G.nodes())
    df['Department'] = pd.Series(nx.get_node_attributes(G,'Department'))
    df['Degrees'] = pd.Series(nx.degree(G))
    df['Clustering'] = pd.Series(nx.clustering(G))
    df['DegreesCentrality'] = pd.Series(nx.degree_centrality(G))
    df['ClosenessCentrality'] = pd.Series(nx.closeness_centrality(G, normalized=True))
    df['BetweennessCentrality'] = pd.Series(nx.betweenness_centrality(G, normalized=True, endpoints=False, k=20))
    df['MgmtSal'] = pd.Series(nx.get_node_attributes(G,'ManagementSalary'))    
    
    # Creating X and y, dropping nan values
    X = df[~df.isnull().any(axis=1)][['Degrees','Clustering','DegreesCentrality','ClosenessCentrality','BetweennessCentrality']]
    y = df[~df.isnull().any(axis=1)]['MgmtSal']
    
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3)
#    scaler = MinMaxScaler()
#    X_train = scaler.fit_transform(X_train)
#    X_test = scaler.transform(X_test)
#    clf = GradientBoostingClassifier().fit(X_train, y_train)
    clf = RandomForestClassifier(n_estimators=100 ,max_features=None, n_jobs=-1).fit(X_train, y_train)
#    RandomForestClassifier --> AUC = 0.75
#    GradientBoostingClassifier --> 0.78

    y_pred = clf.predict(X_test)
    fpr, tpr, _ = roc_curve(y_test, y_pred)
    roc_auc = auc(fpr, tpr)
    print(roc_auc)
    
    predict_proba = clf.predict_proba(df[df.isnull().any(axis=1)][['Degrees','Clustering','DegreesCentrality','ClosenessCentrality','BetweennessCentrality']])

    return pd.Series(predict_proba[:,1], index=df[df.isnull().any(axis=1)].index.values)

salary_predictions()

0.808720691074


1       0.03
2       0.39
5       1.00
8       0.38
14      0.01
18      0.08
27      0.01
30      0.88
31      0.30
34      0.00
37      0.01
40      0.31
45      0.00
54      0.35
55      0.61
60      0.11
62      1.00
65      0.50
77      0.01
79      0.21
97      0.00
101     0.10
103     0.46
108     0.04
113     0.48
122     0.00
141     0.07
142     1.00
144     0.00
145     0.68
        ... 
913     0.00
914     0.00
915     0.00
918     0.10
923     0.00
926     0.05
931     0.00
934     0.00
939     0.00
944     0.00
945     0.00
947     0.20
950     0.04
951     0.00
953     0.00
959     0.00
962     0.00
963     0.06
968     0.00
969     0.00
974     0.01
984     0.00
987     0.00
989     0.00
991     0.00
992     0.00
994     0.00
996     0.00
1000    0.00
1001    0.03
Length: 252, dtype: float64

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [39]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [44]:
def new_connections_predictions():
    
    df_jac = pd.DataFrame(index=[(x[0],x[1]) for x in list(nx.jaccard_coefficient(G))])
    df_jac['JaccardCoef'] = [x[2] for x in list(nx.jaccard_coefficient(G))]
    
    df_res = pd.DataFrame(index=[(x[0],x[1]) for x in list(nx.resource_allocation_index(G))])
    df_res['ResourceAllocation'] = [x[2] for x in list(nx.resource_allocation_index(G))]

    df_pa = pd.DataFrame(index=[(x[0],x[1]) for x in list(nx.preferential_attachment(G))])
    df_pa['PreferentialAttachment'] = [x[2] for x in list(nx.preferential_attachment(G))]

    df_cnsh = pd.DataFrame(index=[(x[0],x[1]) for x in list(nx.cn_soundarajan_hopcroft(G, community='Department'))])
    df_cnsh['CN_SoundarajanHopcroft'] = [x[2] for x in list(nx.cn_soundarajan_hopcroft(G, community='Department'))]

    df_rash = pd.DataFrame(index=[(x[0],x[1]) for x in list(nx.ra_index_soundarajan_hopcroft(G, community='Department'))])
    df_rash['RA_SoundarajanHopcroft'] = [x[2] for x in list(nx.ra_index_soundarajan_hopcroft(G, community='Department'))]
    
    dfs = [df_jac, df_res, df_pa, df_cnsh, df_rash]
    df = dfs[0].join(dfs[1:])
    df = pd.merge(future_connections, df, how='left', left_index=True, right_index=True)
    
    X = df[~df['Future Connection'].isnull()][['JaccardCoef','ResourceAllocation','PreferentialAttachment','CN_SoundarajanHopcroft','RA_SoundarajanHopcroft']]
    y = df[~df['Future Connection'].isnull()]['Future Connection']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    
    clf = RandomForestClassifier(n_estimators=100 ,max_features=None, n_jobs=-1).fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    fpr, tpr, _ = roc_curve(y_test, y_pred)
    roc_auc = auc(fpr, tpr)
    print(roc_auc)
    
    predict_proba = clf.predict_proba(df[df['Future Connection'].isnull()][['JaccardCoef','ResourceAllocation','PreferentialAttachment','CN_SoundarajanHopcroft','RA_SoundarajanHopcroft']])
    
    return pd.Series(predict_proba[:,1], index=df[df['Future Connection'].isnull()].index.values)

new_connections_predictions()

0.810358008318


(107, 348)    0.000000
(542, 751)    0.010257
(20, 426)     0.650000
(50, 989)     0.011342
(942, 986)    0.009410
(324, 857)    0.018833
(13, 710)     0.160000
(19, 271)     0.190000
(319, 878)    0.013250
(659, 707)    0.015144
(49, 843)     0.013063
(208, 893)    0.008957
(377, 469)    0.000000
(405, 999)    0.000000
(129, 740)    0.000000
(292, 618)    0.040000
(239, 689)    0.005228
(359, 373)    0.266543
(53, 523)     0.220000
(276, 984)    0.007069
(202, 997)    0.011864
(604, 619)    0.000000
(270, 911)    0.015407
(261, 481)    0.030000
(200, 450)    1.000000
(213, 634)    0.022520
(644, 735)    0.020000
(346, 553)    0.000000
(521, 738)    0.025157
(422, 953)    0.000000
                ...   
(672, 848)    0.015407
(28, 127)     1.000000
(202, 661)    0.004140
(54, 195)     1.000000
(295, 864)    0.000000
(814, 936)    0.014323
(839, 874)    0.009410
(139, 843)    0.014181
(461, 544)    0.018352
(68, 487)     0.000000
(622, 932)    0.010022
(504, 936)    0.000000
(479, 528) 